In [2]:
import pandas as pd
import numpy as np 
import argparse
import math
import sys

from scipy.stats import pearsonr
import matplotlib.pyplot as plt 

import smlmodule

from itertools import combinations

LIMIT = 0.95

__provmaps__ = {
    "bolzano_bozen": "bolzano",
    "bolzanobozen": "bolzano",
    "vibovalentia": "vibo_valentia",
    "laquila": "l_aquila",
    "laspezia": "la_spezia",
    "barlettaandriatrani": "bat",
    "ascolipiceno": "ascoli_piceno",
    "carboniaiglesias": "carbonia",
    "reggioemilia": "reggio_nell_emilia",
    "pesarourbino": "pesaro",
    "monzabrianza": "monza",
    "reggiocalabria": "reggio_di_calabria",
    "forlicesena": "forli",
    "massacarrara": "massa",
    "verbanocusioossola": "verbania",
    "verbano_cusio_ossola": "verbania",
    "massa_carrara": "massa",
    "monza_e_della_brianza": "monza",
    "pesaro_e_urbino": "pesaro",
    "forli__cesena": "forli",
    "bolzano_/_bozen": "bolzano",
    "barletta_andria_trani": "bat",
    "sud_sardegna": "carbonia",
    "forlì_cesena": "forli"
}

def filterprovname (inprov):
    low = inprov.lower()
    low = low.rstrip()
    low = low.lstrip()
    low = low.replace(" ", "_")
    low = low.replace("'", "_")
    low = low.replace("-", "_")

    return low

def normalize_provname (indata, provcolumn, verbose):

    dict_data = {}  
    for c in indata.columns:
        if verbose:
            print("  ", c)
        if c != provcolumn:
            dict_data[c] = []
    dict_data["prov"] = []

    for i, row in indata.iterrows():
        for c in indata.columns:    
            if c != provcolumn:
                dict_data[c].append(row[c])
            else:
                low = filterprovname(row[c])
                if low in __provmaps__:
                    low = __provmaps__[low]

                dict_data["prov"].append(low)

    #for v in dict_data:
    #    print(v, " ", len(dict_data[v]))

    data = pd.DataFrame.from_dict(dict_data)

    return data



In [12]:
verbose = False
paperpath = "particulate.csv"
agefeatspath = "provinceages.csv"
deprividxpath = "ID11_prov21.xlsx"
tabellecodicipath = "TabelleCodici.xlsx"

tc = pd.ExcelFile(tabellecodicipath)

idtoprov = {}
province = tc.parse("Codice Provincia")
for val in province[["Codice Provincia","Nome Provincia"]].values:
    if type(val[1]) != float:
        idtoprov[int(val[0])] = val[1]
        print(int(val[0]), val[1])

in_datapaper = pd.read_csv(paperpath, sep=";")
in_deprividx =  pd.ExcelFile(deprividxpath).parse("Foglio1")
in_agefeatures = pd.read_csv(agefeatspath)
in_agefeatures = in_agefeatures[in_agefeatures.Population2020 != 0.0]

print("Paper data ")
datapaper = normalize_provname(in_datapaper, "Province", False)
print("Age features ")
agefeatures = normalize_provname(in_agefeatures, "Provincia", False)

1 TORINO
2 VERCELLI
3 NOVARA
4 CUNEO
5 ASTI
6 ALESSANDRIA
7 AOSTA
8 IMPERIA
9 SAVONA
10 GENOVA
11 LA SPEZIA
12 VARESE
13 COMO
14 SONDRIO
15 MILANO
16 BERGAMO
17 BRESCIA
18 PAVIA
19 CREMONA
20 MANTOVA
21 BOLZANO
22 TRENTO
23 VERONA
24 VICENZA
25 BELLUNO
26 TREVISO
27 VENEZIA
28 PADOVA
29 ROVIGO
30 UDINE
31 GORIZIA
32 TRIESTE
33 PIACENZA
34 PARMA
35 REGGIO NELL'EMILIA
36 MODENA
37 BOLOGNA
38 FERRARA
39 RAVENNA
40 FORLI'-CESENA
41 PESARO E URBINO
42 ANCONA
43 MACERATA
44 ASCOLI PICENO
45 MASSA CARRARA
46 LUCCA
47 PISTOIA
48 FIRENZE
49 LIVORNO
50 PISA
51 AREZZO
52 SIENA
53 GROSSETO
54 PERUGIA
55 TERNI
56 VITERBO
57 RIETI
58 ROMA
59 LATINA
60 FROSINONE
61 CASERTA
62 BENEVENTO
63 NAPOLI
64 AVELLINO
65 SALERNO
66 L'AQUILA
67 TERAMO
68 PESCARA
69 CHIETI
70 CAMPOBASSO
71 FOGGIA
72 BARI
73 TARANTO
74 BRINDISI
75 LECCE
76 POTENZA
77 MATERA
78 COSENZA
79 CATANZARO
80 REGGIO DI CALABRIA
81 TRAPANI
82 PALERMO
83 MESSINA
84 AGRIGENTO
85 CALTANISSETTA
86 ENNA
87 CATANIA
88 RAGUSA
89 SIRACUSA
90 SASSAR

In [16]:
dict_deprividx = {}
print("DrepivIdx name ")
for c in in_deprividx.columns:
    if verbose:
        print("   ", c)   
    dict_deprividx[c] = []
dict_deprividx["prov"] = []

for i, row in in_deprividx.iterrows():
    id = row["prov21"]
    prov = filterprovname(idtoprov[id])
    
    if prov in __provmaps__:
        prov = __provmaps__[prov]
    
    #print(id, prov)
    dict_deprividx["prov"].append(prov)
    for c in in_deprividx.columns:
        dict_deprividx[c].append(row[c])


deprividx = pd.DataFrame.from_dict(dict_deprividx)       

DrepivIdx name 


In [17]:
provincelist = list(set(list(datapaper["prov"].values)) & \
        set(list(deprividx["prov"].values)) & \
        set(list(agefeatures["prov"].values)))

print("Province list: ")
for i, p in enumerate(provincelist):
    print("  ", i+1, " ", p)

Province list: 
   1   sondrio
   2   isernia
   3   firenze
   4   genova
   5   perugia
   6   caserta
   7   udine
   8   nuoro
   9   lucca
   10   cosenza
   11   mantova
   12   catanzaro
   13   viterbo
   14   ascoli_piceno
   15   roma
   16   belluno
   17   pavia
   18   cuneo
   19   verona
   20   campobasso
   21   lecco
   22   l_aquila
   23   caltanissetta
   24   imperia
   25   trento
   26   arezzo
   27   brescia
   28   piacenza
   29   milano
   30   savona
   31   bari
   32   verbania
   33   fermo
   34   napoli
   35   modena
   36   vercelli
   37   rimini
   38   bat
   39   novara
   40   salerno
   41   macerata
   42   ferrara
   43   palermo
   44   messina
   45   la_spezia
   46   agrigento
   47   enna
   48   alessandria
   49   carbonia
   50   reggio_nell_emilia
   51   pesaro
   52   frosinone
   53   pescara
   54   taranto
   55   latina
   56   vibo_valentia
   57   livorno
   58   teramo
   59   benevento
   60   pordenone
   61   crotone
   